In [ ]:
#buyer_folios.xlsx final


import pandas as pd
from datetime import datetime

# Load the Excel files
contacts_df = pd.read_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\final_combined_contacts_data.xlsx')
buyer_contacts_df = pd.read_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\buyer_contacts.xlsx')

# Function to clean dollar values (removes $ sign)
def clean_dollar_value(value):
    if pd.isna(value):
        return None
    return value.replace('$', '').strip()

# Function to extract date after 'from' or 'to' in the Agreement column
def extract_agreement_dates(value):
    if pd.isna(value):
        return None, None
    # Split the text by space
    parts = value.split()
    
    # Look for 'from' and 'to' and extract the corresponding dates
    try:
        if 'from' in parts and 'to' in parts:
            from_index = parts.index('from') + 1
            to_index = parts.index('to') + 1
            agreement_start = pd.to_datetime(' '.join(parts[from_index:from_index+3])).strftime('%Y-%m-%d')
            agreement_end = pd.to_datetime(' '.join(parts[to_index:to_index+3])).strftime('%Y-%m-%d')
            return agreement_start, agreement_end
    except Exception as e:
        return None, None
    return None, None

# Function to extract folio code (value inside the first bracket)
def extract_folio_code(value):
    if pd.isna(value):
        return None
    if '(' in value and ')' in value:
        return value.split('(')[1].split(')')[0]
    return None

# Initialize an empty list to store the final data
data = []

# Counter for auto-incremental id starting from 1
auto_id = 1

# Iterate over the rows with non-empty 'Inner_Buyer' in contacts_df
for index, row in contacts_df[contacts_df['Inner_Buyer'].notna()].iterrows():
    
    # Extract agreement_start and agreement_end using the updated function
    agreement_start, agreement_end = extract_agreement_dates(row['Inner_Agreement (Buyer)'])
    
    # Extract asking_price, purchase_price, contract_exchange, deposit_due, settlement_due, and commission
    asking_price = clean_dollar_value(row['Inner_Asking Price'])
    purchase_price = clean_dollar_value(row['Inner_Purchase Price'])
    contract_exchange = pd.to_datetime(row['Inner_Contract Exchange'], errors='coerce').strftime('%Y-%m-%d') if pd.notna(row['Inner_Contract Exchange']) else None
    deposit_due = pd.to_datetime(row['Inner_Deposit Due'], errors='coerce').strftime('%Y-%m-%d') if pd.notna(row['Inner_Deposit Due']) else None
    settlement_due = pd.to_datetime(row['Inner_Settlement'], errors='coerce').strftime('%Y-%m-%d') if pd.notna(row['Inner_Settlement']) else None
    commission = clean_dollar_value(row['Inner_Commission'])
    
    # Find buyer_contact_id by matching 'Inner_Buyer' with 'reference' in buyer_contacts_df
    buyer_contact_id = None
    matched_buyer = buyer_contacts_df[buyer_contacts_df['reference'] == row['Inner_Buyer']]
    if not matched_buyer.empty:
        buyer_contact_id = matched_buyer['id'].values[0]
    
    # Skip rows where buyer_contact_id is empty
    if buyer_contact_id is None:
        continue
    
    # Extract folio_code
    folio_code = extract_folio_code(row['Inner_Folio Code (Buyer)'])
    
    # Append the data for this row
    data.append([
        auto_id,  # id (auto-incremental)
        agreement_start,  # agreement_start
        agreement_end,  # agreement_end
        asking_price,  # asking_price
        purchase_price,  # purchase_price
        contract_exchange,  # contract_exchange
        deposit_due,  # deposit_due
        settlement_due,  # settlement_due
        commission,  # commission
        buyer_contact_id,  # buyer_contact_id
        datetime.now().strftime('%Y-%m-%d %H:%M:%S'),  # created_at (current time)
        datetime.now().strftime('%Y-%m-%d %H:%M:%S'),  # updated_at (current time)
        folio_code,  # folio_code
        1  # company_id (1 for all)
    ])

    # Increment the auto_id for the next row
    auto_id += 1

# Define the columns for the final DataFrame
columns = [
    'id', 'agreement_start', 'agreement_end', 'asking_price', 'purchase_price', 
    'contract_exchange', 'deposit_due', 'settlement_due', 'commission', 
    'buyer_contact_id', 'created_at', 'updated_at', 'folio_code', 'company_id'
]

# Create the DataFrame and save it to Excel
buyer_folios_df = pd.DataFrame(data, columns=columns)
buyer_folios_df.to_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\buyer_folios.xlsx', index=False)

print("buyer_folios.xlsx file has been created successfully, rows with empty buyer_contact_id have been removed.")
